<a href="https://colab.research.google.com/github/Keshav-Sundar-4/RD_ML_Model/blob/main/Oregonator_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

#NEED TO DEFINE TARGET IMAGE, SEED STATE IS INITIAL PARAMS


In [ ]:
#@title Get Images

!wget https://raw.githubusercontent.com/Keshav-Sundar-4/RD_ML_Model/main/Spiral_RD.jpeg


In [ ]:
#@title Image Manipulation

# Load the target image
def load_target_image(image_path, device, resize=None):
    image = Image.open(image_path).convert('RGB')
    if resize:
        image = image.resize(resize, Image.LANCZOS)
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    target_image = transform(image).unsqueeze(0).to(device)
    return target_image

#Convert to Grayscale for improved Pattern extraction
def convert_to_grayscale(image):
    gray_image = 0.2989 * image[:, 0:1, :, :] + \
                 0.5870 * image[:, 1:2, :, :] + \
                 0.1140 * image[:, 2:3, :, :]
    return gray_image


# Load and preprocess target image
image_path = 'Spiral_RD.jpeg'
height, width = 128, 128  # Adjust as needed
target_image = load_target_image(image_path, device, resize=(width, height))
target_image_gray = convert_to_grayscale(target_image)

In [ ]:
#@title Oregonator Model Definition

# Define the model
class OregonatorModel(nn.Module):
    def __init__(self, initial_params, device):
        super(OregonatorModel, self).__init__()
        # Trainable parameters derived from concentrations
        self.epsilon = nn.Parameter(torch.tensor(initial_params['epsilon'], device=device))
        self.f = nn.Parameter(torch.tensor(initial_params['f'], device=device))
        self.q = nn.Parameter(torch.tensor(initial_params['q'], device=device))
        self.D_x = nn.Parameter(torch.tensor(initial_params['D_x'], device=device))
        self.D_z = nn.Parameter(torch.tensor(initial_params['D_z'], device=device))

        self.device = device
        self.laplacian_kernel = self.get_laplacian_kernel().to(device)

    def get_laplacian_kernel(self):
        kernel = torch.tensor([[0.05, 0.2, 0.05],
                               [0.2, -1.0, 0.2],
                               [0.05, 0.2, 0.05]], dtype=torch.float32)
        kernel = kernel.unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, 3, 3)
        return kernel

    def laplacian(self, u):
        u_pad = F.pad(u, (1, 1, 1, 1), mode='circular')  # Wrap-around boundary conditions
        delta_u = F.conv2d(u_pad, self.laplacian_kernel, padding=0)
        return delta_u

    def forward(self, x, z, dt):
        delta_x = self.laplacian(x)
        delta_z = self.laplacian(z)

        # Reaction terms with trainable parameters
        reaction_x = x * (1 - x) - self.f * ((x - self.q) / (x + self.q + 1e-8)) * z
        reaction_z = x - z

        # Update x and z with trainable epsilon and diffusion coefficients
        x = x + dt * (reaction_x / self.epsilon + self.D_x * delta_x)
        z = z + dt * (reaction_z + self.D_z * delta_z)

        return x, z

class InitialConcentrations(nn.Module):
    def __init__(self, height, width, device):
        super(InitialConcentrations, self).__init__()
        # Initialize x and z with random values and make them trainable
        self.x0 = nn.Parameter(torch.rand(1, 1, height, width, device=device))
        self.z0 = nn.Parameter(torch.rand(1, 1, height, width, device=device))

    def forward(self):
        return self.x0, self.z0

def variables_to_image(x, z):
    # Normalize x and z to [0, 1]
    x_norm = (x - x.min()) / (x.max() - x.min() + 1e-8)
    z_norm = (z - z.min()) / (z.max() - z.min() + 1e-8)

    # Combine x and z to create a grayscale image
    gray = 0.5 * (x_norm + z_norm)
    return gray  # Shape: (batch_size, 1, height, width)


In [ ]:
#@title Loss Functions

class TextureLoss(nn.Module):
    def __init__(self, target_image):
        super(TextureLoss, self).__init__()
        self.vgg = models.vgg19(pretrained=True).features[:21].eval()  # Up to 'conv4_1'
        self.vgg.to(device)
        for param in self.vgg.parameters():
            param.requires_grad = False

        # Preprocess target image
        self.target_grams = self.get_grams(target_image)

    def get_features(self, x):
        x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        features = []
        for layer in self.vgg:
            x = layer(x)
            if isinstance(layer, nn.ReLU):
                features.append(x)
        return features

    def gram_matrix(self, feature):
        (b, c, h, w) = feature.size()
        features = feature.view(b, c, h * w)
        G = torch.bmm(features, features.transpose(1, 2))
        G = G / (c * h * w)
        return G

    def get_grams(self, x):
        features = self.get_features(x)
        grams = [self.gram_matrix(f) for f in features]
        return grams

    def forward(self, x):
        x_grams = self.get_grams(x)
        loss = sum(F.mse_loss(g1, g2) for g1, g2 in zip(x_grams, self.target_grams))
        return loss


class CombinedLoss(nn.Module):
    def __init__(self, target_image_gray):
        super(CombinedLoss, self).__init__()
        self.texture_loss = TextureLoss(target_image_gray)

    def forward(self, output_image):
        texture = self.texture_loss(output_image)
        loss = texture
        return loss

In [ ]:
#@title Parameter Intialization

# Initialize model and optimizer
initial_params = {
    'epsilon': 0.02,
    'f': 1.4,
    'q': 0.002,
    'D_x': 1.0,
    'D_z': 0.5
}
model = OregonatorModel(initial_params, device).to(device)
init_conc = InitialConcentrations(height, width, device).to(device)
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(init_conc.parameters()),
    lr=1e-3
)
loss_fn = CombinedLoss(target_image_gray)

In [ ]:
#@title Training Loop

# Training parameters
num_iterations = 1000
time_steps = 100
dt = 1.0
loss_history = []

for iteration in range(num_iterations):
    x_iter, z_iter = init_conc()
    x_iter = torch.clamp(x_iter, 0.0, 1.0)
    z_iter = torch.clamp(z_iter, 0.0, 1.0)

    for _ in range(time_steps):
        x_iter, z_iter = model(x_iter, z_iter, dt)
        x_iter = torch.clamp(x_iter, 0.0, 1.0)
        z_iter = torch.clamp(z_iter, 0.0, 1.0)

    output_image = variables_to_image(x_iter, z_iter)
    loss = loss_fn(output_image)
    loss_value = loss.item()
    loss_history.append(loss_value)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if iteration % 50 == 0:
        print(f"Iteration {iteration}, Loss: {loss_value}")
        output_image_np = output_image.squeeze(0).detach().cpu().numpy()
        plt.imsave(f'output_image_{iteration}.png', output_image_np[0], cmap='gray')

# Plot loss curve
plt.figure()
plt.plot(loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.savefig('loss_curve.png')